In [1]:
import bldw
import glob
import numpy as np
import blimpy as bl
import matplotlib.pyplot as plt
import setigen
import os
%matplotlib inline

# for in-line (non-terminal) turboSETI
# as in Elan Lavie's notebook
import turbo_seti.find_doppler.seti_event as turbo
import turbo_seti.find_event as find
from turbo_seti.find_doppler.find_doppler import FindDoppler
from turbo_seti.find_event.find_event_pipeline import find_event_pipeline
from turbo_seti.find_event.plot_event_pipeline import plot_event_pipeline

In [2]:
DATADIR = '/datag/pipeline/AGBT19B_999_121/blc72_blp02/'
lfiles = ['blc72_guppi_58832_16209_MESSIER031_0057.gpuspec.0000.h5',
'blc72_guppi_58832_16530_HIP2792_0058.gpuspec.0000.h5',
'blc72_guppi_58832_16851_MESSIER031_0059.gpuspec.0000.h5',
'blc72_guppi_58832_17168_HIP3077_0060.gpuspec.0000.h5',
'blc72_guppi_58832_17485_MESSIER031_0061.gpuspec.0000.h5',
'blc72_guppi_58832_17801_HIP3223_0062.gpuspec.0000.h5']

In [5]:
# Uncomment for copying files into scratch directory — hopefully shouldn't be necessary for en-masse runs?

#for file in lfiles:
#    console = f'cp {file} /datax/scratch/benjb/'
#    os.system(console)

In [6]:
# remove DC spikes in absence of turboSETI blank_DC option

data_loc = '/datax/scratch/benjb/'

for i in range(len(lfiles)):
    fb = bl.Waterfall(DATADIR + lfiles[i])
    fb.blank_dc(64)
    fb.write_to_hdf5(f'{data_loc}bDC_{lfiles[i]}')

blimpy.waterfall INFO     Conversion time: 5.83sec
blimpy.waterfall INFO     Conversion time: 5.39sec
blimpy.waterfall INFO     Conversion time: 5.44sec
blimpy.waterfall INFO     Conversion time: 5.47sec
blimpy.waterfall INFO     Conversion time: 5.47sec
blimpy.waterfall INFO     Conversion time: 5.46sec


In [7]:
print('Beginning turboSETI search (FindDoppler version) ...')

for i in range(len(lfiles)):
    doppler = FindDoppler(data_loc + 'bDC_' + lfiles[i],
                      max_drift = 4,
                      snr = 10,       
                      out_dir = data_loc, # This is where the turboSETI output files will be stored.
                      n_coarse_chan = 64,
                      gpu_backend = True
                     )
    doppler.search()

print('Done!')


Beginning turboSETI search (FindDoppler version) ...

turbo_seti version 2.0.18
blimpy version 2.0.11
h5py version 2.10.0

find_doppler.63 INFO     {'DIMENSION_LABELS': array([b'time', b'feed_id', b'frequency'], dtype=object), 'az_start': 0.0, 'data_type': 1, 'fch1': 1876.4648423530161, 'foff': -2.7939677238464355e-06, 'ibeam': 1, 'machine_id': 20, 'nbeams': 1, 'nbits': 32, 'nchans': 67108864, 'nifs': 1, 'rawdatafile': 'guppi_58832_16209_MESSIER031_0057.0000.raw', 'source_name': 'MESSIER031', 'src_dej': <Angle 41.2677 deg>, 'src_raj': <Angle 0.71234667 hourangle>, 'telescope_id': 6, 'tsamp': 18.253611008, 'tstart': 58832.18760416667, 'za_start': 0.0}
find_doppler.63 INFO     File: /datax/scratch/benjb/bDC_blc72_guppi_58832_16209_MESSIER031_0057.gpuspec.0000.h5
 drift rates (min, max): (0.000000, 4.000000)
 SNR: 10.000000

Starting ET search using /datax/scratch/benjb/bDC_blc72_guppi_58832_16209_MESSIER031_0057.gpuspec.0000.h5
find_doppler.63 INFO     Parameters: datafile=/datax/scratch

In [5]:
print('Beginning turboSETI search (command line version) ...')

for i in range(len(lfiles)):
    # Execute turboSETI in the terminal
    console = 'turboSETI ' + DATADIR + lfiles[i] + ' -M 4 -s 10 -o' + '/datax/scratch/benjb/'
    os.system(console)

print('Done!')

Beginning turboSETI search ...
Done!


In [5]:
print('Beginning plotSETI algorithm ...')

console = "plotSETI -f 3 -o " + DATADIR + " " + DATADIR
print(console)
os.system(console)

print('Done!')

Beginning plotSETI algorithm ...
plotSETI -f 3 -o /datax/scratch/benjb/m31_data/ /datax/scratch/benjb/m31_data/
Done!


In [9]:
# Create a simple .lst file of the .h5 files in the data directory
h5_list = sorted(glob.glob(os.path.join(data_loc, '*.h5')))
    
# This writes the .h5 files into a .lst, as required by the find_event_pipeline:
h5_list_path = os.path.join(data_loc,'h5_files.lst')
with open(h5_list_path, 'w') as f:
    for h5_path in h5_list:
        f.write(h5_path + '\n')

# You don't have to print, but it's a good way to check that your list is in the correct order:
with open(h5_list_path, 'r') as f:
    print(f.read())

/datax/scratch/benjb/bDC_blc72_guppi_58832_16209_MESSIER031_0057.gpuspec.0000.h5
/datax/scratch/benjb/bDC_blc72_guppi_58832_16530_HIP2792_0058.gpuspec.0000.h5
/datax/scratch/benjb/bDC_blc72_guppi_58832_16851_MESSIER031_0059.gpuspec.0000.h5
/datax/scratch/benjb/bDC_blc72_guppi_58832_17168_HIP3077_0060.gpuspec.0000.h5
/datax/scratch/benjb/bDC_blc72_guppi_58832_17485_MESSIER031_0061.gpuspec.0000.h5
/datax/scratch/benjb/bDC_blc72_guppi_58832_17801_HIP3223_0062.gpuspec.0000.h5



In [10]:
# Create a simple .lst file of the .dat files in the data directory
dat_list = sorted(glob.glob(os.path.join(data_loc, '*.dat')))
    
# This writes the .dat files into a .lst, as required by the find_event_pipeline:
dat_list_path = os.path.join(data_loc, 'dat_files.lst')
with open(dat_list_path, 'w') as f:
    for dat_path in dat_list:
        f.write(dat_path + '\n')

# You don't have to print, but it's a good way to check that your list is in the correct order:
with open(dat_list_path, 'r') as f:
    print(f.read())

/datax/scratch/benjb/bDC_blc72_guppi_58832_16209_MESSIER031_0057.gpuspec.0000.dat
/datax/scratch/benjb/bDC_blc72_guppi_58832_16530_HIP2792_0058.gpuspec.0000.dat
/datax/scratch/benjb/bDC_blc72_guppi_58832_16851_MESSIER031_0059.gpuspec.0000.dat
/datax/scratch/benjb/bDC_blc72_guppi_58832_17168_HIP3077_0060.gpuspec.0000.dat
/datax/scratch/benjb/bDC_blc72_guppi_58832_17485_MESSIER031_0061.gpuspec.0000.dat
/datax/scratch/benjb/bDC_blc72_guppi_58832_17801_HIP3223_0062.gpuspec.0000.dat



In [12]:
csvf_path = os.path.join(data_loc, 'found_event_table_1.csv')
find_event_pipeline(dat_list_path, 
                    filter_threshold = 1, 
                    number_in_cadence = len(dat_list), 
                    csv_name=csvf_path, 
                    saving=True)


************   BEGINNING FIND_EVENT PIPELINE   **************

Assuming the first observation is an ON
find_event_pipeline: file = bDC_blc72_guppi_58832_16209_MESSIER031_0057.gpuspec.0000.dat, tstart = 58832.18760416667, source_name = MESSIER031
find_event_pipeline: file = bDC_blc72_guppi_58832_16530_HIP2792_0058.gpuspec.0000.dat, tstart = 58832.19131944444, source_name = HIP2792
find_event_pipeline: file = bDC_blc72_guppi_58832_16851_MESSIER031_0059.gpuspec.0000.dat, tstart = 58832.19503472222, source_name = MESSIER031
find_event_pipeline: file = bDC_blc72_guppi_58832_17168_HIP3077_0060.gpuspec.0000.dat, tstart = 58832.1987037037, source_name = HIP3077
find_event_pipeline: file = bDC_blc72_guppi_58832_17485_MESSIER031_0061.gpuspec.0000.dat, tstart = 58832.202372685184, source_name = MESSIER031
find_event_pipeline: file = bDC_blc72_guppi_58832_17801_HIP3223_0062.gpuspec.0000.dat, tstart = 58832.206030092595, source_name = HIP3223
There are 6 total files in the filelist /datax/scratch/

,TopHitNum,DriftRate,SNR,Freq,ChanIndx,FreqStart,FreqEnd,CoarseChanNum,FullNumHitsInRange,FileID,...,MJD,RA,DEC,DELTAT,DELTAF,Hit_ID,status,in_n_ons,RFI_in_range,delta_t
0,1,-0.047832,54.297092,1875.119301,481588,1875.120935,1875.117672,0,290,bDC_blc72_guppi_58832_16209_MESSIER031_0057.gp...,...,58832.187604166669,0h42m44.448s,41d16m03.72s,18.253611,-2.793968,,on_table_1,,,0.0
8,9,-0.019133,13.567498,1784.649578,356107,1784.651213,1784.647950,31,8,bDC_blc72_guppi_58832_16209_MESSIER031_0057.gp...,...,58832.187604166669,0h42m44.448s,41d16m03.72s,18.253611,-2.793968,,on_table_1,,,0.0
16,17,-0.038266,11.953426,1695.420366,835206,1695.422001,1695.418737,61,4,bDC_blc72_guppi_58832_16209_MESSIER031_0057.gp...,...,58832.187604166669,0h42m44.448s,41d16m03.72s,18.253611,-2.793968,,on_table_1,,,0.0
19,20,-0.009566,69.009300,1692.042325,995678,1692.043960,1692.040696,62,1063,bDC_blc72_guppi_58832_16209_MESSIER031_0057.gp...,...,58832.187604166669,0h42m44.448s,41d16m03.72s,18.253611,-2.793968,,on_table_1,,,0.0
21,1,-0.009566,36.290947,1875.119273,481598,1875.120908,1875.117644,0,162,bDC_blc72_guppi_58832_16851_MESSIER031_0059.gp...,...,58832.195034722223,0h42m40.968s,41d14m00.6s,18.253611,-2.793968,,on_table_2,,,642.0
28,8,-0.009566,19.267860,1784.649567,356111,1784.651202,1784.647938,31,13,bDC_blc72_guppi_58832_16851_MESSIER031_0059.gp...,...,58832.195034722223,0h42m40.968s,41d14m00.6s,18.253611,-2.793968,,on_table_2,,,642.0
36,16,-0.019133,13.444082,1695.420344,835214,1695.421978,1695.418715,61,8,bDC_blc72_guppi_58832_16851_MESSIER031_0059.gp...,...,58832.195034722223,0h42m40.968s,41d14m00.6s,18.253611,-2.793968,,on_table_2,,,642.0
38,18,-0.019133,41.404341,1692.042320,995680,1692.043954,1692.040691,62,74,bDC_blc72_guppi_58832_16851_MESSIER031_0059.gp...,...,58832.195034722223,0h42m40.968s,41d14m00.6s,18.253611,-2.793968,,on_table_2,,,642.0
47,8,-0.009566,20.723237,1784.649562,356113,1784.651196,1784.647933,31,11,bDC_blc72_guppi_58832_17485_MESSIER031_0061.gp...,...,58832.202372685184,0h42m44.448s,41d15m39.24s,18.253611,-2.793968,,on_table_3,,,1276.0
48,9,0.009566,95.258403,1758.396918,315116,1758.398553,1758.395290,40,583,bDC_blc72_guppi_58832_17485_MESSIER031_0061.gp...,...,58832.202372685184,0h42m44.448s,41d15m39.24s,18.253611,-2.793968,,on_table_3,,,1276.0


In [12]:
# and finally we plot
plot_event_pipeline(csvf_path, # full path of the CSV file built by find_event_pipeline()
                    h5_list_path, # full path of text file containing the list of .h5 files
                    filter_spec='f{}'.format(2), # filter threshold
                    user_validation=False) # Non-interactive

plot_event_pipeline: Opened file /datax/scratch/benjb/m31_data/found_event_table.csv
plot_event_pipeline: file = blc71_guppi_58832_16209_MESSIER031_0057.gpuspec.0000.h5, tstart = 58832.18760416667, source_name = MESSIER031
plot_event_pipeline: file = blc71_guppi_58832_16530_HIP2792_0058.gpuspec.0000.h5, tstart = 58832.19131944444, source_name = HIP2792
plot_event_pipeline: file = blc71_guppi_58832_16851_MESSIER031_0059.gpuspec.0000.h5, tstart = 58832.19503472222, source_name = MESSIER031
plot_event_pipeline: file = blc71_guppi_58832_17168_HIP3077_0060.gpuspec.0000.h5, tstart = 58832.1987037037, source_name = HIP3077
plot_event_pipeline: file = blc71_guppi_58832_17485_MESSIER031_0061.gpuspec.0000.h5, tstart = 58832.202372685184, source_name = MESSIER031
plot_event_pipeline: file = blc71_guppi_58832_17801_HIP3223_0062.gpuspec.0000.h5, tstart = 58832.206030092595, source_name = HIP3223
Plotting some events for:  MESSIER031
There are 2 total events in the csv file /datax/scratch/benjb/m31_